In [18]:
%pip install PyPDF2
%pip install sentence-transformers
%pip install load_dotenv
%pip install azure-search-documents
%pip install azure-ai-openai
from tenacity import retry, wait_random_exponential, stop_after_attempt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement azure-ai-openai (from versions: none)
ERROR: No matching distribution found for azure-ai-openai


In [19]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import json

In [20]:
FILE_PATH = "./pdf/biology.pdf"

In [21]:
def get_pdf_data(file_path, num_pages = 1):
    reader = PdfReader(file_path)
    full_doc_text = ""
    pages = reader.pages
    num_pages = len(pages) 
    
    try:
        for page in range(num_pages):
            current_page = reader.pages[page]
            text = current_page.extract_text()
            full_doc_text += text
    except:
        print("Error reading file")
    finally:
        return full_doc_text

In [22]:
# Divide the text into chunks of chunk_length 
# [ default is 500] characters
def get_chunks(fulltext:str,chunk_length =500) -> list:
    text = fulltext

    chunks = []
    while len(text) > chunk_length:
        last_period_index = text[:chunk_length].rfind('.')
        if last_period_index == -1:
            last_period_index = chunk_length
        chunks.append(text[:last_period_index])
        text = text[last_period_index+1:]
    chunks.append(text)

    return chunks

In [23]:
filename = FILE_PATH

In [24]:
FILE_PATH.split('/')[-1]

'biology.pdf'

In [25]:
full_doc_text = get_pdf_data(filename)

In [26]:
print(f'Full doc text length: {len(full_doc_text)}')

Full doc text length: 31720


In [27]:
Lines =get_chunks(full_doc_text,500)

In [28]:
len(Lines)


73

In [29]:
type(Lines)

list

In [30]:
from openai import AzureOpenAI
import os

In [31]:
load_dotenv()

True

In [58]:
client = AzureOpenAI(
  api_key = "CX64A1mGMPNCogk5dL9E1BhDnyUmh6q64rsWtfISCcBtTpvkJO2XJQQJ99BLACHYHv6XJ3w3AAAAACOGPdqx", #os.getenv("AZURE_OPENAI_KEY"),  
  api_version = "2024-12-01-preview", #os.getenv("AZURE_API_VERSION"),
  azure_endpoint = "https://foundrychunkingrepo.cognitiveservices.azure.com/" #os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [59]:
model: str = "text-embedding-3-small" #os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_ID")
print(f'Using embedding model: {model}')

Using embedding model: text-embedding-3-small


In [60]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text, model=model):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [61]:
counter = 0

In [62]:
input_data = []

In [63]:
for line in Lines:
    d = {}
    d['id'] = str(counter)
    d['line'] = line
    d['embedding'] = generate_embeddings(line)
    d['filename'] = FILE_PATH.split('/')[-1]
    counter = counter + 1
    input_data.append(d)

In [64]:
counter

73

In [65]:
len(input_data[0]['embedding'])

1536

In [66]:
# Output embeddings to docVectors.json file
with open("./output/docVectors_azure.json", "w") as f:
    json.dump(input_data, f)